In [0]:
#upgrading the tensor flow to version 2.1.0
!pip install tensorflow==2.1.0

In [0]:
!pip install flask-ngrok

In [3]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from flask import Flask,request,render_template,redirect,url_for
#from werkzeug.utils import secure_filename

import os
import time
from threading import Timer
from PIL import Image
import pickle
import glob
from flask_ngrok import run_with_ngrok

In [5]:
# For changing working directory in Google Colab
#%cd /content/drive/My\ Drive/Colab\ Notebooks/unit7_project
#!pwd

/content/drive/My Drive/Colab Notebooks/unit7_project
/content/drive/My Drive/Colab Notebooks/unit7_project


In [0]:
app=Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
app.config['UPLOAD_FOLDER'] = os.path.join('/static')

# Loading recommendation images
bracelet_glob=sorted(glob.glob("static/bracelet/*.jpg"))
earring_glob=sorted(glob.glob("static/earring/*.jpg"))
necklace_glob=sorted(glob.glob("static/necklace/*.jpg"))
ring_glob=sorted(glob.glob("static/ring/*.jpg"))

# Loading binary files of models
if 'model' not in globals():
  model=tf.keras.models.load_model('best_model_accuracy.h5')

bracelet_cluster=pickle.load(open('bracelet_cluster.pkl','rb'))
if 'bracelet_encoder' not in globals():
  bracelet_encoder=tf.keras.models.load_model('bracelet_encoder.h5')
earring_cluster=pickle.load(open('earring_cluster.pkl', 'rb'))
if 'earring_encoder' not in globals():
  earring_encoder=tf.keras.models.load_model('earring_encoder.h5')
necklace_cluster=pickle.load(open('necklace_cluster.pkl','rb'))
if 'necklace_encoder' not in globals():
  necklace_encoder=tf.keras.models.load_model('necklace_encoder.h5')
ring_cluster=pickle.load(open('ring_cluster.pkl','rb'))
if 'ring_encoder' not in globals():
  ring_encoder=tf.keras.models.load_model('ring_encoder.h5')

dict1={0: 'a bracelet',
       1: 'earrings',
       2: 'a necklace',
       3: 'a ring'}

@app.route('/jwl_recom/index.html',methods=['GET','POST'])
def load_index():
    return render_template('index.html')

@app.route('/jwl_recom/result.html', methods=['GET'])
def result_get_request():
    return redirect(url_for('load_index'))

@app.route('/jwl_recom')
def load_index_empty_1():
    return redirect(url_for('load_index'))

@app.route('/')
def load_index_empty_2():
    return redirect(url_for('load_index'))

@app.route('/jwl_recom/')
def load_index_empty_3():
    return redirect(url_for('load_index'))

@app.route('/jwl_recom/result.html', methods=['POST'])
def uploader():

    global file,img,counter
    counter=0
    file = request.files['file']
    file.save(file.filename)
    new_filename=str(int(time.time()))+".jpg"
    img=Image.open(file.filename)
    if img.format not in ("JPEG","JPEG 2000"):
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        img.save(new_filename)
        os.remove(file.filename)
    else:
        img.close()
        os.rename(file.filename,new_filename)

    img=tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(new_filename,target_size=(200,200)))/255.0
    img=img[np.newaxis,:,:,:]
    jwl_type=dict1[model.predict_classes(img)[0]]
    result=model.predict_classes(img)[0]
    Timer(5,remove_fun,args=[new_filename]).start()
    return render_template('result.html',jwl_type=jwl_type,
                           br_checked=jwl_checked(result)[0],
                           ea_checked=jwl_checked(result)[1],
                           ne_checked=jwl_checked(result)[2],
                           ri_checked=jwl_checked(result)[3])

@app.route('/jwl_recom/recom.html', methods=['POST'])
def recom():
  global counter,jwl_type,recom_type
  if counter==0:    
      jwl_type=request.form['jewelry_type']
      recom_type=request.form['recom_type']
      recom_img=img_path(img,jwl_type,recom_type)
      full_filename = os.path.join("/", recom_img)
      counter+=1
      return render_template('recom.html',jwl_type=jwl_type, image_url=full_filename)
  else:
      recom_img=img_path(img,jwl_type,recom_type)
      full_filename = os.path.join("/", recom_img)
      counter+=1
      return render_template('recom.html',jwl_type=jwl_type, image_url=full_filename)         

def jwl_checked(output_type):
    if output_type==0:
        return "checked","","",""
    elif output_type==1:
        return "","checked","",""
    elif output_type==2:
        return "","","checked",""
    else:
        return "","","","checked"
    
def img_path(image,jwl_type,recom_type):
    if jwl_type=='bracelet' and recom_type=='similar':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=bracelet_encoder.predict(image_flat)
        image_cluster=bracelet_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(bracelet_cluster.labels_==image_cluster)[0])
        return bracelet_glob[random_image]

    if jwl_type=='bracelet' and recom_type=='different':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=bracelet_encoder.predict(image_flat)
        image_cluster=bracelet_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(bracelet_cluster.labels_!=image_cluster)[0])
        return bracelet_glob[random_image] 

    if jwl_type=='earrings' and recom_type=='similar':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=earring_encoder.predict(image_flat)
        image_cluster=earring_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(earring_cluster.labels_==image_cluster)[0])
        return earring_glob[random_image]

    if jwl_type=='earrings' and recom_type=='different':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=earring_encoder.predict(image_flat)
        image_cluster=earring_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(earring_cluster.labels_!=image_cluster)[0])
        return earring_glob[random_image]

    if jwl_type=='necklace' and recom_type=='similar':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=necklace_encoder.predict(image_flat)
        image_cluster=necklace_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(necklace_cluster.labels_==image_cluster)[0])
        return necklace_glob[random_image]   

    if jwl_type=='necklace' and recom_type=='different':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=necklace_encoder.predict(image_flat)
        image_cluster=necklace_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(necklace_cluster.labels_!=image_cluster)[0])
        return necklace_glob[random_image]    

    if jwl_type=='ring' and recom_type=='similar':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=ring_encoder.predict(image_flat)
        image_cluster=ring_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(ring_cluster.labels_==image_cluster)[0])
        return ring_glob[random_image]

    if jwl_type=='ring' and recom_type=='different':
        image_flat=image.reshape(image.shape[0],-1)
        encoded_img=ring_encoder.predict(image_flat)
        image_cluster=ring_cluster.predict(encoded_img)
        random_image=np.random.choice(np.where(ring_cluster.labels_!=image_cluster)[0])
        return ring_glob[random_image]       

        
def remove_fun(filename):
    os.remove(filename)
    
if __name__ == '__main__':
    app.run()
